In [1]:
import re

def modify(text):
    if '[ⓒ' in text:
        index = text.find('[ⓒ')
        text = text[:index]
    while 'googletag.display' in text:
        index = text.find('googletag.display')        
        index2 = text.find(';',index)
        text = text[:index]+text[index2+1:]
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' #이메일 주소 제거
    text = re.sub(pattern=pattern, repl=' ',string=text)
    
    return text
def modify2(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' #한글 자모음 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[^\w\s]' #특수기호 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    return text

In [2]:
import re
import csv
f = open('final_articles.csv','r',encoding='utf-8-sig')
rdf = csv.reader(f)
total = []
for line in rdf:
    string = modify(line[2])
    split = re.split('(?<=[^0-9])[\.|\n]', string) #마침표 기준으로 나누기(소수점은 마침표 취급 x)/본문 뒤 [ㅇㅇㅇ기자]~ 제거 =>기사전문
    if split == []:
        continue
    split = [modify(s).strip() for s in split]
    total.append((split,line[1]))

In [3]:
from konlpy.tag import Komoran
from textrank import KeywordSummarizer
from textrank import KeysentenceSummarizer

komoran = Komoran()

def komoran_tokenize(sent):
    return komoran.pos(sent, join=True)

result = []
n =0
for sents in total:
    n +=1
    try:
        summarizer = KeysentenceSummarizer(tokenize = komoran_tokenize, min_sim = 0.3)
        keysents = summarizer.summarize(sents[0], topk=int(min(len(sents[0]),max(3,len(sents[0])*6/10))))
        sentense = ''
        for line in keysents:
            sentense+=line[2]
        result.append([sents[1],sentense])
    except Exception as e:
        print(n,', ' ,e)

639 ,  row, column, and data array must all be the same length
640 ,  row, column, and data array must all be the same length
642 ,  row, column, and data array must all be the same length
643 ,  row, column, and data array must all be the same length
644 ,  row, column, and data array must all be the same length
882 ,  row, column, and data array must all be the same length
883 ,  row, column, and data array must all be the same length
884 ,  row, column, and data array must all be the same length
1543 ,  row, column, and data array must all be the same length
1544 ,  row, column, and data array must all be the same length
1547 ,  row, column, and data array must all be the same length
1549 ,  row, column, and data array must all be the same length
1550 ,  row, column, and data array must all be the same length
1551 ,  row, column, and data array must all be the same length
1552 ,  row, column, and data array must all be the same length
1783 ,  row, column, and data array must all be 

In [4]:
import csv
f = open('final_articles_sum.csv','w', newline='',encoding='utf-8-sig')
wr = csv.writer(f)
for line in result:
    if '삼성전자' in line[1]:
        wr.writerow([None]+line)
f.close()